# Import and Store NNDC Deposition Data - First Objective

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time

### Setup

The [radlist](https://www.nndc.bnl.gov/radlist/radlist2.jsp) API that parses the ENSDF Data needs the latter as its payload. To the user, merely knowing the nuclide and decay radiation parameters should be enough, as on [Decay Radiation Search](https://www.nndc.bnl.gov/nudat3/indx_dec.jsp), instead of copying values from one place to another.

On searching for the decay data of any nuclide on the webpage above, we get another page that holds the link to ENSDF data.

Therefore, we first parse this HTML response to fetch the `href` link to ENSDF Data using `BeautifulSoup`.

### Fetch the ENSDF Data Download Link

The payload for the API has been retrieved by inspecting the 'Network Activity' when sending a POST request to the URL.

In [2]:
def get_ensdf_link(nuclide):

    url = "https://www.nndc.bnl.gov/nudat3/dec_searchi.jsp"
    # default payload for the decay-search API
    data = {
        "spnuc": "name",
        "nuc": nuclide,
        "z": "",
        "a": "",
        "n": "",
        "zmin": "",
        "zmax": "",
        "amin": "",
        "amax": "",
        "nmin": "",
        "nmax": "",
        "evenz": "any",
        "evena": "any",
        "evenn": "any",
        "tled": "disabled",
        "tlmin": 0,
        "utlow": "FS",
        "tlmax": 1E10,
        "utupp": "GY",
        "dmed": "disabled",
        "dmn": "ANY",
        "rted": "disabled",
        "rtn": "ANY",
        "reed": "disabled",
        "remin": 0,
        "remax": 10000,
        "ried": "disabled",
        "rimin": 0,
        "rimax": 200,
        "ord": "zate",
        "out": "wp",
        "unc": "nds",
        "sub": "Search"
    }

    result = []
    
    #Send POST request
    response = requests.post(url, data=data)

    result.append(response.status_code)

    if response.status_code == 200:
        # Convert the response to string
        html_text = response.text

        # parse the HTML string through Beautiful Soup
        soup = BeautifulSoup(html_text, 'html.parser')

        links = soup.find_all('a')

        # The SECOND hyperlink on the webpage is ENSDF Download Link
        result.append(links[1].get('href'))
    else:
        result.append(response.reason)

    return result

### Fetch the ENSDF data with the link retrieved above through a GET request

In [3]:
def get_ensdf_data_from_link(nuclide):
    
    decay_link = get_ensdf_link(nuclide)
    if(decay_link[0] == 200):
        ensdf_link = "https://www.nndc.bnl.gov/nudat3/" + decay_link[1]
        
        response = requests.get(ensdf_link)
        
        soup = BeautifulSoup(response.text,'html.parser')
        ensdf_data = soup.find('pre').text
        
        return ensdf_data
    
    else:
        print (f"{decay_link[1]}. Error Code: {decay_link[0]}")

### Convert and Write to a JSON file after a POST request to 'radlist'

In [6]:
def get_json_from_ensdf(nuclide):

    ensdf_data = get_ensdf_data_from_link(nuclide)
    
    #Preparing the data to be parsed by 'radlist' as EXACTLY one leading whitespace is allowed.
    ensdf_data = " " + ensdf_data.strip()
    
    # Set the URL and data to send
    url = "https://www.nndc.bnl.gov/radlist/radlist2.jsp"
    data = {
        "mydata": ensdf_data,
        "format": "json",
        "submit": "Submit"
    }
 
    # Send the POST request. 'requests' converts data into the accepted Content-Type: x-www-form-urlencoded.
    response = requests.post(url, data=data)
    
    if response.status_code == 200:
        # Save the JSON response to a file
        print (response.json())
        
        #Write to file and Download
        with open(f'{nuclide}_json_data.json', 'w') as f:
            json.dump(response.json(), f)
    else:
        print(f"{response.reason}. Error Code:{response.status_code}")

### Enter the nuclide

In [7]:
nuclide = "Co56"
get_json_from_ensdf(nuclide)

{'parent': {'a': '56', 'decayMode': {'name': 'EC', 'uncertainty': '', 'value': '100'}, 'level': {'jpis': '4+', 'energy': {'uncertainty': '', 'value': '0.0'}}, 'z': '27', 'halflife': {'inSeconds': '6673190.4', 'units': 'D', 'uncertainty': '26', 'value': '77.236'}, 'qValue': {'uncertainty': '20', 'value': '4566.0'}, 'n': '29', 'element': 'Co'}, 'radiation': {'betaPlusAV': [{'dose': '0.121', 'radIntensityUncertainty': '14', 'radIntensity': '19.7', 'energyUncertainty': '6', 'subType': '', 'epEnergyUncertainty': '', 'type': 'bp av', 'energy': '6.1E+2', 'epEnergy': '', 'doseUncertainty': '15'}], 'electron': [{'dose': '7.44E-4', 'radIntensityUncertainty': '6', 'radIntensity': '111.0', 'energyUncertainty': '', 'subType': 'Auger L', 'epEnergyUncertainty': '', 'type': 'e', 'energy': '0.67', 'epEnergy': '', 'doseUncertainty': '4'}, {'dose': '0.002605', 'radIntensityUncertainty': '4', 'radIntensity': '46.3', 'energyUncertainty': '', 'subType': 'Auger K', 'epEnergyUncertainty': '', 'type': 'e', 'en